# Fit Neg Binomial model 

Taking inspiration from [GLM.jl](https://github.com/JuliaStats/GLM.jl/blob/master/src/negbinfit.jl#L68), we will:
+ Initialize `r` with Poisson regression fit
+ Perform block updates:
    - Fix $r$, fit negative binomial copula until convergence
    - Fix $\beta$ and variance compoenent parameters, fit $r$ using Newton
    - Repeat until convergence
    
# No block updates (only initialize $r$)

In [28]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 100
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [35.0, 32.0, 49.0, 37.0, 53.0]
 [55.0, 40.0, 39.0, 45.0, 35.0]
 [34.0, 42.0, 43.0, 60.0, 34.0]
 [41.0, 39.0, 34.0, 43.0, 55.0]
 [25.0, 60.0, 26.0, 32.0, 53.0]
 [42.0, 41.0, 46.0, 44.0, 47.0]
 [58.0, 47.0, 45.0, 42.0, 43.0]
 [48.0, 52.0, 39.0, 37.0, 56.0]
 [43.0, 33.0, 38.0, 39.0, 38.0]
 [44.0, 44.0, 39.0, 37.0, 36.0]
 [34.0, 36.0, 45.0, 35.0, 39.0]
 [39.0, 50.0, 51.0, 32.0, 42.0]
 [34.0, 47.0, 36.0, 50.0, 37.0]
 ⋮
 [49.0, 46.0, 53.0, 42.0, 34.0]
 [54.0, 46.0, 48.0, 36.0, 49.0]
 [49.0, 54.0, 38.0, 45.0, 32.0]
 [36.0, 47.0, 53.0, 54.0, 45.0]
 [34.0, 39.0, 42.0, 41.0, 59.0]
 [35.0, 38.0, 37.0, 49.0, 48.0]
 [35.0, 46.0, 53.0, 29.0, 34.0]
 [30.0, 56.0, 48.0, 46.0, 40.0]
 [40.0, 47.0, 35.0, 38.0, 48.0]
 [45.0, 37.0, 37.0, 28.0, 40.0]
 [29.0, 35.0, 53.0, 44.0, 46.0]
 [47.0, 48.0, 54.0, 38.0, 47.0]

In [29]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [30]:
initialize_model!(gcm)
@show gcm.β
# @show gcm.Σ
@show gcm.r

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -183965.880323221 39999
2 -183965.880323221 -183965.880323221 39999
initializing variance components using MM-Algorithm
gcm.Σ = [225.63027521074963, 1857.7441889422705]
gcm.θ = [-5.404042280046205, 224.7201241053222, 1864.4405333056088]
gcm.θ = [3.76335082451623, 225.63027521074963, 1857.7441889422705]
gcm.θ = [3.76335082451623, 225.63027521074963, 1857.7441889422705]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.

* grad_f[          2] = -1.1624128938060323e+01    ~ -1.1610857440697405e+01  [ 1.143e-03]
* grad_f[          3] =  1.4000239622658115e+00    ~  1.3984520467760686e+00  [ 1.124e-03]

Derivative checker detected 2 error(s).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros

1-element Vector{Float64}:
 64.78937959114738

In [31]:
β_true

3.757872325600888

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [32]:
GLMCopula.loglikelihood!(gcm, true, true)

-178909.94656519336

## fitting routine

In [33]:
# Quasi-Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100,
#     tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",
#     mehrotra_algorithm="yes", warm_start_init_point="yes",
    hessian_approximation = "limited-memory"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

-178909.59176759253

In [34]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.7611786562926066; true β = 3.757872325600888
estimated variance component 1 = 0.04829707124136521; true variance component 1 = 0.1
estimated variance component 2 = 0.04440991310090281; true variance component 2 = 0.5
estimated r = 64.78937959114738; true r = 100


In [27]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.456226847936331; true β = 1.4552872326068422
estimated variance component 1 = 0.10133551134062899; true variance component 1 = 0.1
estimated variance component 2 = 0.4371874111348784; true variance component 2 = 0.5
estimated r = 9.668018779781647; true r = 10


# Block updates

In [74]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 1000
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [405.0, 395.0, 407.0, 445.0, 463.0]
 [388.0, 435.0, 439.0, 437.0, 405.0]
 [402.0, 428.0, 426.0, 372.0, 453.0]
 [432.0, 418.0, 401.0, 426.0, 469.0]
 [482.0, 372.0, 479.0, 461.0, 393.0]
 [427.0, 425.0, 440.0, 433.0, 413.0]
 [478.0, 444.0, 437.0, 428.0, 430.0]
 [447.0, 397.0, 439.0, 411.0, 385.0]
 [430.0, 399.0, 415.0, 417.0, 414.0]
 [423.0, 423.0, 439.0, 444.0, 449.0]
 [455.0, 448.0, 419.0, 452.0, 439.0]
 [418.0, 453.0, 401.0, 461.0, 427.0]
 [454.0, 443.0, 408.0, 453.0, 412.0]
 ⋮
 [450.0, 440.0, 462.0, 428.0, 454.0]
 [465.0, 440.0, 446.0, 448.0, 449.0]
 [449.0, 391.0, 415.0, 420.0, 460.0]
 [449.0, 444.0, 394.0, 466.0, 420.0]
 [454.0, 439.0, 428.0, 425.0, 482.0]
 [451.0, 415.0, 446.0, 450.0, 447.0]
 [452.0, 416.0, 394.0, 387.0, 402.0]
 [389.0, 384.0, 446.0, 439.0, 436.0]
 [436.0, 413.0, 452.0, 442.0, 409.0]
 [437.0, 444.0, 412.0, 383.0, 435.0]
 [386.0, 452.0, 463.0, 423.0, 440.0]
 [413.0, 446.0, 392.0, 441.0, 444.0]

In [75]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [76]:
initialize_model!(gcm)
@show gcm.β
# @show gcm.Σ
@show gcm.r

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -241695.7820575578 39999
2 -241695.7820575578 -241695.7820575578 39999
initializing variance components using MM-Algorithm
gcm.Σ = [205.1754518638664, 1794.1380547738995]
gcm.θ = [-3.1062653307058063, 204.265300758439, 1800.8343991372378]
gcm.θ = [6.061127773856629, 205.1754518638664, 1794.1380547738995]
gcm.θ = [6.061127773856629, 205.1754518638664, 1794.1380547738995]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.

* grad_f[          2] = -1.2694500526824070e+01    ~ -1.2766256428482533e+01  [ 5.621e-03]
* grad_f[          3] =  1.4388060636195519e+00    ~  1.4778412665241274e+00  [ 2.641e-02]

Derivative checker detected 2 error(s).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzer

1-element Vector{Float64}:
 567.5406480971419

In [77]:
β_true

6.060457418594933

In [78]:
GLMCopula.loglikelihood!(gcm, true, true)

-236675.90934349902

In [79]:
# Quasi-Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100,
#     tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",
#     mehrotra_algorithm="yes", warm_start_init_point="yes",
    hessian_approximation = "limited-memory"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_nb.jl:33



This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_nb.jl:33


This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

...

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_nb.jl:33



This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_nb.jl:33


This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

-236626.29268013974

In [80]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 6.060603531823474; true β = 6.060457418594933
estimated variance component 1 = 0.0935902801194254; true variance component 1 = 0.1
estimated variance component 2 = 0.4507832268881768; true variance component 2 = 0.5
estimated r = 977.723922064876; true r = 1000


In [73]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 3.758435534322703; true β = 3.757872325600888
estimated variance component 1 = 0.0934850410967605; true variance component 1 = 0.1
estimated variance component 2 = 0.3827355454688751; true variance component 2 = 0.5
estimated r = 94.45211633725886; true r = 100
